<h1>ETL Project</h1>

<p>We were tasked with obtaining a data set and applying Extract, Transform and Load (ETL).  We chose NYPD Complaints data and NY Zip Code with Latitude and Longitude data.  The data sets were obtained from the following websites:</p>

* https://catalog.data.gov/dataset/nypd-complaint-data-current-year-to-date
* https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/

<p>Though we were not required to run any analysis on the data, we decided to work with NYPD Complaints data because the dataset offered several years (>10 years) which would allow for some interesting analysis later. Still we narrowed the scope during the Transform process and focused only on July and December 2019 & 2020 data.</p>

<p>Some of the challenges we experienced were joining the 2 datasets based on Latitude and Longitude coordinates which were not the same.  This caused confidence issues because the results were duplicated for some records
Also, coding the database and table creation in SQLAlchemy within the Jupyter Notebook and pushing to PGADMIN was something new we never attempted.</p>

In [1]:
import pandas as pd
from pandas import DataFrame
from datetime import date, timedelta
from dotenv import load_dotenv, find_dotenv
import os
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, Float, DateTime, Date, MetaData
from sqlalchemy.sql import select
from sqlalchemy import join
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

### Extract CSVs into DataFrames

In [2]:
crime_file = "./Resources/2019_NYPD_Complaint_Data.csv"
crime_df = pd.read_csv(crime_file)
crime_df.head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_TO_DT,CRM_ATPT_CPTD_CD,KY_CD,LAW_CAT_CD,OFNS_DESC,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Latitude,Longitude
0,192701448,QUEENS,10/7/1918,2/2/2019,COMPLETED,109,FELONY,GRAND LARCENY,UNKNOWN,UNKNOWN,U,45-64,WHITE,M,40.762070,-73.929744
1,122423264,STATEN ISLAND,11/24/1919,11/24/2019,COMPLETED,109,FELONY,GRAND LARCENY,NaN,NaN,NaN,45-64,WHITE,M,40.538354,-74.215574
2,595267268,MANHATTAN,1/9/1920,1/9/2020,COMPLETED,341,MISDEMEANOR,PETIT LARCENY,UNKNOWN,BLACK,U,UNKNOWN,UNKNOWN,D,40.722053,-73.988215
3,759752818,MANHATTAN,4/29/1920,4/29/2020,COMPLETED,341,MISDEMEANOR,PETIT LARCENY,UNKNOWN,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,D,40.731890,-73.994546
4,351329583,QUEENS,7/6/1920,7/7/2020,COMPLETED,341,MISDEMEANOR,PETIT LARCENY,45-64,ASIAN / PACIFIC ISLANDER,M,18-24,BLACK HISPANIC,M,40.758563,-73.865846


In [3]:
# NYPD Complaint Data
complaint_file = "./Resources/2020_NYPD_Complaint_Data.csv"
NYPD_data = pd.read_csv(complaint_file)
NYPD_data.head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_TO_DT,CRM_ATPT_CPTD_CD,KY_CD,LAW_CAT_CD,OFNS_DESC,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Latitude,Longitude
0,192701448,QUEENS,10/7/1918,2/2/2019,COMPLETED,109,FELONY,GRAND LARCENY,UNKNOWN,UNKNOWN,U,45-64,WHITE,M,40.762070,-73.929744
1,122423264,STATEN ISLAND,11/24/1919,11/24/2019,COMPLETED,109,FELONY,GRAND LARCENY,NaN,NaN,NaN,45-64,WHITE,M,40.538354,-74.215574
2,595267268,MANHATTAN,1/9/1920,1/9/2020,COMPLETED,341,MISDEMEANOR,PETIT LARCENY,UNKNOWN,BLACK,U,UNKNOWN,UNKNOWN,D,40.722053,-73.988215
3,759752818,MANHATTAN,4/29/1920,4/29/2020,COMPLETED,341,MISDEMEANOR,PETIT LARCENY,UNKNOWN,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,D,40.731890,-73.994546
4,351329583,QUEENS,7/6/1920,7/7/2020,COMPLETED,341,MISDEMEANOR,PETIT LARCENY,45-64,ASIAN / PACIFIC ISLANDER,M,18-24,BLACK HISPANIC,M,40.758563,-73.865846


In [4]:
lat_long = "./Resources/us_zip_lat_long.csv"
lat_long_df = pd.read_csv(lat_long)
lat_long_df.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,55795,Willow River,MN,46.317812,-92.84315,-6,1,"46.317812, -92.84315"
1,45388,Yorkshire,OH,40.328535,-84.47938,-5,1,"40.328535, -84.47938"
2,39483,Foxworth,MS,31.218509,-89.90761,-6,1,"31.218509, -89.90761"
3,31503,Waycross,GA,31.205194,-82.37534,-5,1,"31.205194, -82.37534"
4,45833,Delphos,OH,40.841409,-84.34178,-5,1,"40.841409, -84.34178"


### Transform 2019 Crime DataFrame

<p>Transforming the NYPD Complaints data entailed renaming the column headings to a more specific title.  I also rounded the Latitude and Longitude columns to 1, 2 and 3 digits which helped later during the join query in SQLAlchemy. Finally I dropped any NaN values and formatted the date columns to be datetime format rather than a string.  This change helped in narrowing the dataset so that only July and December 2019 data showed.</p>

In [5]:
# Create a filtered dataframe from specific columns
crime_cols = ["CMPLNT_NUM", "BORO_NM", "CMPLNT_FR_DT", "CMPLNT_TO_DT", "CRM_ATPT_CPTD_CD", "KY_CD", "LAW_CAT_CD",
                "OFNS_DESC", "SUSP_AGE_GROUP", "SUSP_RACE", "SUSP_SEX", "VIC_AGE_GROUP", 
                "VIC_RACE", "VIC_SEX", "Latitude", "Longitude"]

crime_transformed = crime_df[crime_cols].copy()

# Rename the column headers
crime_transformed = crime_transformed.rename(columns={"CMPLNT_NUM": "Complaint_ID", 
                                                      "BORO_NM": "Boro_Name", 
                                                      "CMPLNT_FR_DT":"Start_Date", 
                                                      "CMPLNT_TO_DT":"End_Date", 
                                                      "CRM_ATPT_CPTD_CD":"Complaint_Status", 
                                                      "KY_CD":"Complaint_Code", 
                                                      "LAW_CAT_CD": "Complaint_Cat",
                                                      "OFNS_DESC":"Complaint_Desc", 
                                                      "SUSP_AGE_GROUP":"Suspect_Age", 
                                                      "SUSP_RACE":"Suspect_Race", 
                                                      "SUSP_SEX": "Suspect_Gender", 
                                                      "VIC_AGE_GROUP":"Victim_Age", 
                                                      "VIC_RACE":"Victim_Race", 
                                                      "VIC_SEX":"Victim_Gender", 
                                                      "Latitude":"Latitude", 
                                                      "Longitude":"Longitude"})

#Convert the Start Date and End Date columns from object to datetime datatypes
crime_transformed["Start_Date"] = pd.to_datetime(crime_transformed["Start_Date"], format="%m/%d/%Y", errors="coerce")
crime_transformed["End_Date"] = pd.to_datetime(crime_transformed["End_Date"], format="%m/%d/%Y", errors="coerce")

#Filter to return where the start date and end date year is equal to 2019
crime_transformed_filter=crime_transformed[(crime_transformed["Start_Date"].dt.year == 2019) & (crime_transformed["End_Date"].dt.year == 2019)]

#Filter to return where the month equals July or December based on the start date
crime_transformed_filter=crime_transformed_filter[(crime_transformed_filter["Start_Date"].dt.month_name() == "July") | (crime_transformed_filter["Start_Date"].dt.month_name() == "December")]

#This will show only the rows that have a NAN value to allow the user the option of dropping them.
crime_transformed_filter_na = crime_transformed_filter[crime_transformed_filter.isna().any(axis=1)]
crime_transformed_filter_na

cols=["Latitude", "Longitude"]
crime_transformed_filter[["Lat3", "Long3"]] = crime_transformed_filter[cols].round(3)
crime_transformed_filter[["Lat2", "Long2"]] = crime_transformed_filter[cols].round(2)
crime_transformed_filter[["Lat1", "Long1"]] = crime_transformed_filter[cols].round(1)

crime_transformed_filter.dropna()
crime_transformed_filter

,Complaint_ID,Boro_Name,Start_Date,End_Date,Complaint_Status,Complaint_Code,Complaint_Cat,Complaint_Desc,Suspect_Age,Suspect_Race,...,Victim_Race,Victim_Gender,Latitude,Longitude,Lat3,Long3,Lat2,Long2,Lat1,Long1
2845,968286251,BRONX,2019-07-01,2019-09-01,COMPLETED,116,FELONY,SEX CRIMES,<18,UNKNOWN,...,BLACK HISPANIC,F,40.887451,-73.847608,40.887,-73.848,40.89,-73.85,40.9,-73.8
2846,374736401,MANHATTAN,2019-07-01,2019-08-31,COMPLETED,109,FELONY,GRAND LARCENY,UNKNOWN,UNKNOWN,...,WHITE,M,40.781198,-73.959618,40.781,-73.960,40.78,-73.96,40.8,-74.0
2847,737315396,BRONX,2019-07-01,2019-08-31,COMPLETED,104,FELONY,RAPE,<18,BLACK,...,WHITE HISPANIC,F,40.853960,-73.900121,40.854,-73.900,40.85,-73.90,40.9,-73.9
2848,498947462,BRONX,2019-07-01,2019-09-30,COMPLETED,116,FELONY,SEX CRIMES,25-44,UNKNOWN,...,UNKNOWN,M,40.856163,-73.844397,40.856,-73.844,40.86,-73.84,40.9,-73.8
2849,489982024,BRONX,2019-07-01,2019-09-01,COMPLETED,233,MISDEMEANOR,SEX CRIMES,<18,BLACK,...,BLACK HISPANIC,M,40.887451,-73.847608,40.887,-73.848,40.89,-73.85,40.9,-73.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8502,204105951,BROOKLYN,2019-12-31,2019-12-31,COMPLETED,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,UNKNOWN,BLACK,...,BLACK,M,40.685332,-73.980704,40.685,-73.981,40.69,-73.98,40.7,-74.0
8503,116431516,MANHATTAN,2019-12-31,2019-12-31,COMPLETED,341,MISDEMEANOR,PETIT LARCENY,45-64,BLACK,...,UNKNOWN,D,40.762810,-73.962748,40.763,-73.963,40.76,-73.96,40.8,-74.0
8506,441666491,MANHATTAN,2019-12-31,2019-12-31,COMPLETED,578,VIOLATION,HARRASSMENT 2,UNKNOWN,UNKNOWN,...,BLACK,F,40.711068,-73.990959,40.711,-73.991,40.71,-73.99,40.7,-74.0
8508,211965127,QUEENS,2019-12-31,2019-12-31,COMPLETED,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,18-24,BLACK,...,BLACK,M,40.686612,-73.781801,40.687,-73.782,40.69,-73.78,40.7,-73.8


### Transform 2020 Crime DataFrame

In [6]:
NYPD_df = NYPD_data

NYPD_df["CMPLNT_FR_DT"] = pd.to_datetime(NYPD_df["CMPLNT_FR_DT"], format="%m/%d/%Y", errors="coerce")
NYPD_df["CMPLNT_TO_DT"] = pd.to_datetime(NYPD_df["CMPLNT_TO_DT"], format="%m/%d/%Y", errors="coerce")

# Get the year 2020
startdate = "1/1/2020"
enddate = "12/31/2020"

after_start_date_DT = NYPD_df["CMPLNT_FR_DT"] >= startdate
before_end_date_DT = NYPD_df["CMPLNT_FR_DT"] <= enddate
between_two_dates = after_start_date_DT & before_end_date_DT
NYPD_2020_df = NYPD_df[between_two_dates]

NYPD_2020_new_df = NYPD_2020_df.rename(columns={'CMPLNT_NUM' : 'Complaint_ID', 
                                                'BORO_NM' : 'Boro_Name', 
                                                'CMPLNT_FR_DT': 'Start_Date', 
                                                'CMPLNT_TO_DT': 'End_Date', 
                                                'CRM_ATPT_CPTD_CD': 'Complaint_Status',
                                                'KY_CD':'Complaint_Code', 
                                                'LAW_CAT_CD':'Complaint_Cat', 
                                                'SUSP_AGE_GROUP' :'Suspect_Age', 
                                                'SUSP_RACE' : 'Suspect_Race', 
                                                'SUSP_SEX' : 'Suspect_Gender', 
                                                'VIC_AGE_GROUP' : 'Victim_Age', 
                                                'VIC_RACE' : 'Victim_Race',
                                                'VIC_SEX' : 'Victim_Gender', 
                                                'OFNS_DESC' : 'Complaint_Desc'  })

NYPD_2020_2m_df= NYPD_2020_new_df[(NYPD_2020_new_df["Start_Date"].dt.month_name() == "July") | (NYPD_2020_new_df["Start_Date"].dt.month_name() == "December")]

NYPD_2020_2m_df[["Lat3", "Long3"]] = NYPD_2020_2m_df[cols].round(3)
NYPD_2020_2m_df[["Lat2", "Long2"]] = NYPD_2020_2m_df[cols].round(2)
NYPD_2020_2m_df[["Lat1", "Long1"]] = NYPD_2020_2m_df[cols].round(1)

NYPD_2020_2m_df

C:\Users\bxprd\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Complaint_ID,Boro_Name,Start_Date,End_Date,Complaint_Status,Complaint_Code,Complaint_Cat,Complaint_Desc,Suspect_Age,Suspect_Race,...,Victim_Race,Victim_Gender,Latitude,Longitude,Lat3,Long3,Lat2,Long2,Lat1,Long1
203886,396232931,STATEN ISLAND,2020-07-01,2020-12-28,COMPLETED,126,FELONY,MISCELLANEOUS PENAL LAW,UNKNOWN,BLACK,...,BLACK,F,40.635748,-74.121447,40.636,-74.121,40.64,-74.12,40.6,-74.1
203887,169917183,BRONX,2020-07-01,NaT,COMPLETED,109,FELONY,GRAND LARCENY,45-64,WHITE HISPANIC,...,WHITE HISPANIC,F,40.831587,-73.876532,40.832,-73.877,40.83,-73.88,40.8,-73.9
203888,921274840,STATEN ISLAND,2020-07-01,2020-12-09,COMPLETED,361,MISDEMEANOR,OFF. AGNST PUB ORD SENSBLTY &,UNKNOWN,WHITE,...,WHITE,F,40.585646,-74.103297,40.586,-74.103,40.59,-74.10,40.6,-74.1
203889,474695098,STATEN ISLAND,2020-07-01,2020-12-20,COMPLETED,109,FELONY,GRAND LARCENY,NaN,NaN,...,BLACK,F,40.628223,-74.157257,40.628,-74.157,40.63,-74.16,40.6,-74.2
203890,910399654,QUEENS,2020-07-01,2020-12-15,COMPLETED,578,VIOLATION,HARRASSMENT 2,UNKNOWN,BLACK,...,WHITE HISPANIC,F,40.690480,-73.779437,40.690,-73.779,40.69,-73.78,40.7,-73.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413398,988706831,BRONX,2020-12-31,2020-12-31,COMPLETED,121,FELONY,CRIMINAL MISCHIEF & RELATED OF,UNKNOWN,UNKNOWN,...,WHITE HISPANIC,M,40.873780,-73.826107,40.874,-73.826,40.87,-73.83,40.9,-73.8
413399,780729893,MANHATTAN,2020-12-31,NaT,COMPLETED,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,25-44,BLACK,...,BLACK HISPANIC,F,40.807864,-73.935851,40.808,-73.936,40.81,-73.94,40.8,-73.9
413400,694763592,MANHATTAN,2020-12-31,2020-12-31,COMPLETED,578,VIOLATION,HARRASSMENT 2,UNKNOWN,BLACK,...,BLACK,F,40.792412,-73.972428,40.792,-73.972,40.79,-73.97,40.8,-74.0
413401,203039918,BRONX,2020-12-31,2020-12-31,COMPLETED,578,VIOLATION,HARRASSMENT 2,18-24,BLACK,...,BLACK,F,40.854431,-73.905589,40.854,-73.906,40.85,-73.91,40.9,-73.9


In [7]:
# DropNA
NYPD_2020_2m_df= NYPD_2020_2m_df.dropna()
NYPD_2020_2m_df

,Complaint_ID,Boro_Name,Start_Date,End_Date,Complaint_Status,Complaint_Code,Complaint_Cat,Complaint_Desc,Suspect_Age,Suspect_Race,...,Victim_Race,Victim_Gender,Latitude,Longitude,Lat3,Long3,Lat2,Long2,Lat1,Long1
203886,396232931,STATEN ISLAND,2020-07-01,2020-12-28,COMPLETED,126,FELONY,MISCELLANEOUS PENAL LAW,UNKNOWN,BLACK,...,BLACK,F,40.635748,-74.121447,40.636,-74.121,40.64,-74.12,40.6,-74.1
203888,921274840,STATEN ISLAND,2020-07-01,2020-12-09,COMPLETED,361,MISDEMEANOR,OFF. AGNST PUB ORD SENSBLTY &,UNKNOWN,WHITE,...,WHITE,F,40.585646,-74.103297,40.586,-74.103,40.59,-74.10,40.6,-74.1
203890,910399654,QUEENS,2020-07-01,2020-12-15,COMPLETED,578,VIOLATION,HARRASSMENT 2,UNKNOWN,BLACK,...,WHITE HISPANIC,F,40.690480,-73.779437,40.690,-73.779,40.69,-73.78,40.7,-73.8
203891,219313826,QUEENS,2020-07-01,2020-11-19,COMPLETED,344,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,UNKNOWN,WHITE HISPANIC,...,WHITE,F,40.707139,-73.903259,40.707,-73.903,40.71,-73.90,40.7,-73.9
203893,533416507,MANHATTAN,2020-07-01,2020-07-15,COMPLETED,109,FELONY,GRAND LARCENY,UNKNOWN,UNKNOWN,...,BLACK,M,40.770827,-73.992611,40.771,-73.993,40.77,-73.99,40.8,-74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413397,254894727,BROOKLYN,2020-12-31,2020-12-31,COMPLETED,578,VIOLATION,HARRASSMENT 2,25-44,ASIAN / PACIFIC ISLANDER,...,ASIAN / PACIFIC ISLANDER,M,40.640527,-74.013297,40.641,-74.013,40.64,-74.01,40.6,-74.0
413398,988706831,BRONX,2020-12-31,2020-12-31,COMPLETED,121,FELONY,CRIMINAL MISCHIEF & RELATED OF,UNKNOWN,UNKNOWN,...,WHITE HISPANIC,M,40.873780,-73.826107,40.874,-73.826,40.87,-73.83,40.9,-73.8
413400,694763592,MANHATTAN,2020-12-31,2020-12-31,COMPLETED,578,VIOLATION,HARRASSMENT 2,UNKNOWN,BLACK,...,BLACK,F,40.792412,-73.972428,40.792,-73.972,40.79,-73.97,40.8,-74.0
413401,203039918,BRONX,2020-12-31,2020-12-31,COMPLETED,578,VIOLATION,HARRASSMENT 2,18-24,BLACK,...,BLACK,F,40.854431,-73.905589,40.854,-73.906,40.85,-73.91,40.9,-73.9


### Transform US Zip Code - Latitude/Longitude Data

<p>Transforming the US Zip Code-Latitude/Longitude data was straight forward and only entailed 3 changes:</p>

* Adding 6 columns which were rounded versions of the Latitude and Longitude columns.  Rounding to 1, 2 and 3 digits.
* Dropping unnecessary columns.
* Filtering the data so only NY Zip Codes and Latitude/Longitude showed.

In [8]:
#Round the latitude & longitude columns to 3, 2 & 1 digits and add those columns to the dataframe
lat_long_df[["Lat3", "Long3"]] = lat_long_df[cols].round(3)
lat_long_df[["Lat2", "Long2"]] = lat_long_df[cols].round(2)
lat_long_df[["Lat1", "Long1"]] = lat_long_df[cols].round(1)

#Drop the timezone and Daylight Savings time columns
lat_long_df.drop(["Timezone", "Daylight savings time flag"],axis=1, inplace=True)

#Filter data so only NY state shows
lat_long_NY_df = lat_long_df[(lat_long_df.State.eq("NY"))]

lat_long_NY_df

,Zip,City,State,Latitude,Longitude,geopoint,Lat3,Long3,Lat2,Long2,Lat1,Long1
12,13758,East Pharsalia,NY,42.583518,-75.721931,"42.583518, -75.721931",42.584,-75.722,42.58,-75.72,42.6,-75.7
24,11757,Lindenhurst,NY,40.690049,-73.374420,"40.690049, -73.37442",40.690,-73.374,40.69,-73.37,40.7,-73.4
64,13310,Bouckville,NY,42.888210,-75.568800,"42.88821, -75.5688",42.888,-75.569,42.89,-75.57,42.9,-75.6
94,14812,Beaver Dams,NY,42.286649,-76.976600,"42.286649, -76.9766",42.287,-76.977,42.29,-76.98,42.3,-77.0
150,13562,Hermon,NY,44.465307,-75.230195,"44.465307, -75.230195",44.465,-75.230,44.47,-75.23,44.5,-75.2
...,...,...,...,...,...,...,...,...,...,...,...,...
43169,12569,Pleasant Valley,NY,41.746469,-73.804140,"41.746469, -73.80414",41.746,-73.804,41.75,-73.80,41.7,-73.8
43171,14302,Niagara Falls,NY,43.326840,-78.830681,"43.32684, -78.830681",43.327,-78.831,43.33,-78.83,43.3,-78.8
43174,10303,Staten Island,NY,40.629448,-74.162390,"40.629448, -74.16239",40.629,-74.162,40.63,-74.16,40.6,-74.2
43176,12594,Wingdale,NY,41.646584,-73.562090,"41.646584, -73.56209",41.647,-73.562,41.65,-73.56,41.6,-73.6


### Create database and connections

<p>Creating the database and connections without going through PGADMIN was challenging because it required multiple connections to the engine and several imports for SQLAlchemy.  We also connected to PGADMIN via a .env file containing a connection string to postgres username and password.  The informatation is ignored when committing to Git by putting the file name in the .gitignore file.</p>

In [9]:
#Create connection to Postgres
load_dotenv()
my_env_var = os.getenv('DB_CONNECTION_STRING')
engine = create_engine(my_env_var)

#Create connection to the engine and create nypd_crimes database
connection = engine.connect()
connection.execute("commit")
connection.execute("create database nypd_crimes")

#Point the engine to the newly created nypd_crimes database
engine = create_engine(f'{my_env_var}/nypd_crimes')
connection = engine.connect()

In [10]:
Base = declarative_base
meta = MetaData()

#Create the crimes table 
ny_crimes = Table("crimes", meta,
    Column("Complaint_ID", Integer, primary_key=True),
    Column("Boro_Name", String(255)),
    Column("Start_Date", DateTime),
    Column("End_Date", DateTime),
    Column("Complaint_Status", String(255)),
    Column("Complaint_Code", Integer),
    Column("Complaint_Cat", String(255)),
    Column("Complaint_Desc", String(255)),
    Column("Suspect_Age", String(255)),
    Column("Suspect_Race", String(255)),
    Column("Suspect_Gender", String(255)),
    Column("Victim_Age", String(255)),
    Column("Victim_Race", String(255)),
    Column("Victim_Gender",String(255)),
    Column("Latitude", Float),
    Column("Longitude", Float),
    Column("Lat3", Float),
    Column("Long3", Float),
    Column("Lat2", Float),
    Column("Long2", Float),
    Column("Lat1", Float),
    Column("Long1", Float),)

In [11]:
#Create the Zip Code-Latitude/Longitude table 
ny_lat_long = Table("lat_long", meta,
                    Column("Zip", Integer, primary_key=True),
                    Column("City", String(255)),
                    Column("State",String(255)),
                    Column("Latitude", Float),
                    Column("Longitude", Float),
                    Column("geopoint", String(255)),
                    Column("Lat3", Float),
                    Column("Long3", Float),
                    Column("Lat2", Float),
                    Column("Long2", Float),
                    Column("Lat1", Float),
                    Column("Long1", Float),)

#Create both tables in PGADMIN
meta.create_all(engine)

### Load DataFrames into database

In [12]:
#Push the dataframes to the newly created tables
crime_transformed_filter.to_sql(name="crimes", con = engine, if_exists="replace", index=False)
NYPD_2020_2m_df.to_sql(name="crimes", con = engine, if_exists = "append", index=False)
lat_long_NY_df.to_sql(name="lat_long", con = engine, if_exists="replace", index=False)

In [14]:
# Confirm tables
engine.table_names()

['crimes', 'lat_long']

In [15]:
#Confirm the columns in the crimes table are accurate using inspect
inspector=inspect(engine)
columns=inspector.get_columns("crimes")
for c in columns:
    print(c["name"], c["type"])

Complaint_ID BIGINT
Boro_Name TEXT
Start_Date TIMESTAMP WITHOUT TIME ZONE
End_Date TIMESTAMP WITHOUT TIME ZONE
Complaint_Status TEXT
Complaint_Code BIGINT
Complaint_Cat TEXT
Complaint_Desc TEXT
Suspect_Age TEXT
Suspect_Race TEXT
Suspect_Gender TEXT
Victim_Age TEXT
Victim_Race TEXT
Victim_Gender TEXT
Latitude DOUBLE PRECISION
Longitude DOUBLE PRECISION
Lat3 DOUBLE PRECISION
Long3 DOUBLE PRECISION
Lat2 DOUBLE PRECISION
Long2 DOUBLE PRECISION
Lat1 DOUBLE PRECISION
Long1 DOUBLE PRECISION


In [16]:
#Confirm the columns in the lat_long table are accurate using inspect
inspector=inspect(engine)
columns=inspector.get_columns("lat_long")
for c in columns:
    print(c["name"], c["type"])

Zip BIGINT
City TEXT
State TEXT
Latitude DOUBLE PRECISION
Longitude DOUBLE PRECISION
geopoint TEXT
Lat3 DOUBLE PRECISION
Long3 DOUBLE PRECISION
Lat2 DOUBLE PRECISION
Long2 DOUBLE PRECISION
Lat1 DOUBLE PRECISION
Long1 DOUBLE PRECISION


In [28]:
#Create query to join the 2 tables based on the Latitude and Longitude rounded to the 2 digit columns
j = ny_crimes.join(ny_lat_long, (ny_crimes.c.Lat2 == ny_lat_long.c.Lat2) & (ny_crimes.c.Long2 == ny_lat_long.c.Long2))
stmt = select([ny_crimes, ny_lat_long]).select_from(j)
result = connection.execute(stmt)
merged_df = pd.DataFrame(result.fetchall())
merged_df.columns = result.keys()
merged_df = merged_df[["Complaint_ID", "Boro_Name", "Start_Date", "End_Date", "Complaint_Code", 
                       "Complaint_Cat", "Complaint_Desc", "Suspect_Age", "Suspect_Race", "Suspect_Gender", 
                       "Victim_Age", "Victim_Race", "Victim_Gender", "Latitude", "Longitude", 
                       "Lat2", "Long2", "Zip", "City", "State"]]

In [30]:
#Display the joined dataframe
merged_df.sort_values(by ="Start_Date")

,Complaint_ID,Boro_Name,Start_Date,End_Date,Complaint_Code,Complaint_Cat,Complaint_Desc,Suspect_Age,Suspect_Race,Suspect_Gender,...,Latitude,Longitude,Longitude,Lat2,Lat2,Long2,Long2,Zip,City,State
33435,379046132,MANHATTAN,2019-07-01,2019-10-23,109,FELONY,GRAND LARCENY,25-44,ASIAN / PACIFIC ISLANDER,M,...,40.755101,-73.992598,-73.993370,40.76,40.76,-73.99,-73.99,10018,New York,NY
56648,737315396,BRONX,2019-07-01,2019-08-31,104,FELONY,RAPE,<18,BLACK,M,...,40.846745,-73.900121,-73.898610,40.85,40.85,-73.90,-73.90,10457,Bronx,NY
18723,562915290,BROOKLYN,2019-07-01,2019-12-05,104,FELONY,RAPE,45-64,WHITE,M,...,40.641436,-74.022464,-74.015740,40.64,40.64,-74.02,-74.02,11220,Brooklyn,NY
26420,482261442,MANHATTAN,2019-07-01,2019-07-01,233,MISDEMEANOR,SEX CRIMES,None,None,None,...,40.715721,-73.997491,-74.003950,40.72,40.72,-74.00,-74.00,10278,New York,NY
57423,489982024,BRONX,2019-07-01,2019-09-01,233,MISDEMEANOR,SEX CRIMES,<18,BLACK,M,...,40.890950,-73.847608,-73.847020,40.89,40.89,-73.85,-73.85,10466,Bronx,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19316,840263641,BROOKLYN,2020-12-31,2020-12-31,106,FELONY,FELONY ASSAULT,25-44,BLACK,M,...,40.645099,-73.954492,-73.945032,40.65,40.65,-73.95,-73.95,11247,Brooklyn,NY
19315,840263641,BROOKLYN,2020-12-31,2020-12-31,106,FELONY,FELONY ASSAULT,25-44,BLACK,M,...,40.645099,-73.954492,-73.945032,40.65,40.65,-73.95,-73.95,11241,Brooklyn,NY
55301,906292520,BRONX,2020-12-31,2020-12-31,126,FELONY,MISCELLANEOUS PENAL LAW,45-64,BLACK,F,...,40.829676,-73.906487,-73.908560,40.83,40.83,-73.91,-73.91,10456,Bronx,NY
19322,840263641,BROOKLYN,2020-12-31,2020-12-31,106,FELONY,FELONY ASSAULT,25-44,BLACK,M,...,40.645099,-73.954492,-73.945032,40.65,40.65,-73.95,-73.95,11240,Brooklyn,NY
